<a href="https://colab.research.google.com/github/focussed/etivity1/blob/master/lab2_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab 2: Data Preparation

In the exploration of the *loans* dataset, we discovered a few issues, which need to be resolved before the dataset is ready for machine learning (ML). This exercise is typically referred as either *data preparation*, or *data preprocessing*, or *data munging*, or *data wrangling*. 

Here are the problems, we are already aware of:

- There are missing values in some columns. We can estimate these values depending on the number of missing values and the expected importance of the column.

- We observed that `ApplicantIncome` and `LoanAmount` seem to contain extreme values at either end. Although they might make intuitive sense, they should be treated appropriately.

## A. Import Python Modules and Dataset

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
%matplotlib inline

url_train = 'https://raw.githubusercontent.com/focussed/etivity1/master/loans_train.csv'
df = pd.read_csv(url_train)
df.tail()

FileNotFoundError: ignored

Check for missing values.

In [0]:
df.isna().sum()

Before we start transforming the dataset let's make a copy of it.

In [0]:
df_original = df

## B. Process and Encode the Categorical Features

### Column `Loan_ID`

Columns with ID numbers are unlikely to contain useful information and should be dropped.

In [0]:
df.drop('Loan_ID', axis=1, inplace=True)
df.tail()

### Encode Ordinal Categorical Attributes 

Note that there is only one categorical column, the column `Dependents`, which is **ordinal**. All other categorical columns (after deleting `Loan_ID`) are **nominal**.

Let's first encode `Dependents`. While `Dependents` does look like a numerical column, it is not, because one of the possible values '3+' which is not a number. Still, there is an inherent order among the possible values in this column, i.e. 0 < 1 < 2 < 3+. Thus, we can encode it by replacing '3+' by 3.

In [0]:
# Create a mapper

scale_mapper = {
    "0": 0,
    "1": 1,
    "2": 2,
    "3+": 3
}

df['Dependents'] = df['Dependents'].replace(scale_mapper)

df.tail()

### One-Hot Encode All Other Categorical Attributes

In [0]:
# first one-hot encode the categorical columns with NaNs

df = pd.get_dummies(df, columns=['Gender', 'Married', 'Self_Employed'], 
                        dummy_na=True, 
                        drop_first=True)
df.tail()

In [0]:
# now one-hot encode all other categorical columns

df = pd.get_dummies(df, drop_first=True)
df.head()

## C. Treatment of Missing Values in Numerical Columns

### Check for Missing Values

It is often useful to impute missing values as many ML algorithms do not work with missing data and even if they do, imputing these values often helps to build a more accurate predictive model.

In [0]:
# Find the number of nulls/NaNs in the dataset

df.isna().sum()

Note that missing values may not always be NaNs. For instance, if `Loan_Amount_Term` had zeros, would they make sense or should they be considered missing? That is, we need to understand the dataset well in order to identify which values are missing.

Note also that no column has more than 25% missing values. Thus, we do not need to consider dropping whole columns.

### Impute Missing Values

There are numerous ways to impute (i.e., fill in) the missing values in a numerical column, e.g. column `LoanAmount` – the simplest being replacement by mean, which can be done with the following code:

```pyton
df['LoanAmount'].fillna(df['LoanAmount'].mean(), inplace=True)
```

Imputing missing values with the mean (alternatively, the media, the most frequent, or a constant) may introduce bias in the dataset. A better method is to build an ML model (typically, kNN) to predict `LoanAmount` on the basis of other columns.

Since, the purpose now is to practice data munging prior applying ML algorithms, we can take an approach, which lies somewhere in between these 2 extremes. We may hypothesise that attributes `Education` and `Self_Employed` combined can give a good estimate of `LoanAmount`.

First, let us look at the following boxplots to see if a trend exists:

In [0]:
df.boxplot(column='LoanAmount', by=['Education_Not Graduate','Self_Employed_Yes'], rot=45)
plt.title("")
plt.show()

We can see some variation in the median of `LoanAmount` for each group and these medians can be used for imputing the missing values in each group. Next, we create a pivot table, which contains the median values for all four groups.

In [0]:
ptable = df.pivot_table(values='LoanAmount', 
                        index='Self_Employed_Yes',
                        columns='Education_Not Graduate',  
                        aggfunc=np.median)
ptable

Finally, we define a function, which returns the values in the pivot table and apply it to fill the missing values of `LoanAmount`.

In [0]:
# Define function to return an element of the pivot table
def get_element(x):
    return ptable.loc[x['Self_Employed_Yes'], x['Education_Not Graduate']]

# Replace missing values
df['LoanAmount'].fillna(df[df['LoanAmount'].isnull()].apply(get_element, axis=1), inplace=True)

Let's take a simplified approach for `Dependents`, `Loan_Amount_Term` and `Credit_History` and fill in the missing values in these columns with the median. Note that the median is likely a better choice than the mean for these columns because the means are not likely to be realistic values.

In [0]:
df['Dependents'].fillna(df['Dependents'].median(), inplace=True)
df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].median(), inplace=True)
df['Credit_History'].fillna(df['Credit_History'].median(), inplace=True)

## D. Rescaling Features

Many ML algorithms assume that all numerical features are on the same scale. Two standard techniques in the Python module `scikit-learn` for achieving this are:
* MinMaxScaler - rescales a column to the interval [0,1]
* StandardScaler - rescales a numerical column so that it has mean 0 and standard deviation 1.

Here we use a third technique, called `RobustScaler` to `LoanAmount` and `ApplicantIncome` which does a better job in the presence of outliers (we observed outliers in columns `LoanAmount` and `ApplicantIncome`). `RobustScaler` transforms each value in a column by subtracting the median from it and then dividing by the interquartile range.

We will deal with `CoapplicantIncome` and `Loan_Amount_Term` later. Note that all other attributes are binary.

In [0]:
robust_scaler = RobustScaler()
df[['ApplicantIncome',
    'LoanAmount']] = robust_scaler.fit_transform(df[['ApplicantIncome',
                                                     'LoanAmount']])

To bring column `Dependents` to the interval $[0,1]$ we can simply divide it by 3, as it is an ordinal columns with four possible values: 0, 1, 2 and 3.

In [0]:
df['Dependents'] = df['Dependents']/3

## E. Treatment of Outliers

#### `ApplicantIncome` and `LoanAmount`

In the previous lab exercise, we observed that `ApplicantIncome` and `LoanAmount` contain some extreme values. Let's take a look at the box plots of `ApplicantIncome`,`LoanAmount` to verify that they have outliers. We will deal with `CoapplicantIncome` and `Loan_Amount_Term` later. All other columns do not have outliers.

In [0]:
df.boxplot(column=['ApplicantIncome','LoanAmount'])
plt.show()

Let's also take a look at their histograms after we have rescaled them.

In [0]:
# Create figure with two subplots
fig = plt.figure(figsize=(16,4))

# Plot ApplicantIncome
ax1 = fig.add_subplot(1, 2, 1)
ax1.set_title("Histogram of ApplicantIncome")
ax1.set_xlabel('ApplicantIncome')
ax1.set_ylabel('Number of Applicants')
df['ApplicantIncome'].hist(bins=20)

# Plot LoanAmount
ax3 = fig.add_subplot(1, 2, 2)
ax3.set_title("Histogram of LoanAmount")
ax3.set_xlabel('LoanAmount')
ax3.set_ylabel('Number of Applicants')
df['LoanAmount'].hist(bins=20)

plt.show()

Both histograms are skewed to the left, which is not ideal. Many ML algorithms work best when the features are not skewed to either side.

Finally, before treating the outliers, let's find the minimum values in these columns.

In [0]:
df[['ApplicantIncome', 'LoanAmount']].min()

Since the extreme values are practically possible in both columns, i.e. some people might have large income and also apply for high-value loans, instead of ignoring the data rows with extreme values we can transform them to diminish the negative impact of outliers to ML models. A typical way to do this is to apply a log transformation. However, we do have negative values in both columns after rescaling and we need to add a constant to all values in order to shift them to a positive range before applying a log transformation. 

Here we are have chosen to shift the values to the interval $[1, +\infty]$ before applying a log transformation.

In [0]:
# Use a log transformation to decrease the impact of outliers
df['ApplicantIncome'] = np.log(df['ApplicantIncome']+2.26)
df['LoanAmount'] = np.log(df['LoanAmount']+2.85)

Let's plot again.

In [0]:
# Create figure with two subplots
fig = plt.figure(figsize=(16,4))

# Plot ApplicantIncome
ax1 = fig.add_subplot(1, 2, 1)
ax1.set_title("Histogram of ApplicantIncome Transformed")
ax1.set_xlabel('ApplicantIncome')
ax1.set_ylabel('Number of Applicants')
df['ApplicantIncome'].hist(bins=20)

# Plot LoanAmount
ax3 = fig.add_subplot(1, 2, 2)
ax3.set_title("Histogram of LoanAmount Transformed")
ax3.set_xlabel('LoanAmount')
ax3.set_ylabel('Number of Applicants')
df['LoanAmount'].hist(bins=20)

plt.show()

The histograms show that the transformed features are less skewed and have distributions closer to normal than the original features `LoanAmount` and `ApplicantIncome`.

#### `Loan_Amount_Term`

Let's examine the distribution of values in `Loan_Amount_Term`. 

In [0]:
df['Loan_Amount_Term'].value_counts()

Note that `Loan_Amount_Term` is rather an ordinal than a continuous numerical feature with more than 80% of its values being 360.0. None of its values appear to be actual outliers. To bring it to the interval $[0,1]$ we can divide all values by 480 (effectively applying MinMaxScaler to it). If we applied RobustScaler and/or log transformation to it instead then it would become even more skewed to the right (you may verify this).

In [0]:
df['Loan_Amount_Term'] = df['Loan_Amount_Term']/480
df['Loan_Amount_Term'].hist(bins=20)
plt.show()

## F. Creation of Derived Features

Column `CoapplicantIncome` contains many zeros which can be interpreted as `there is no co-applicant with income greater than 0`. The zeros are *missing values* which should not be filled in because they are missing for a reason. On the other hand, they skew the feature towards 0 and this cannot be easily fixed with a log transformation.

In [0]:
df['CoapplicantIncome'].hist(bins=20)
plt.show()

In [0]:
np.log(df['CoapplicantIncome']+1).hist(bins=20)
plt.show()

What we can do in this case instead is:
* Create a new feature `TotalIncome` as the sum of the *original* `ApplicantIncome` and `CoapplicantIncome`
* Drop column `CoapplicantIncome` because it can be derived from `ApplicantIncome` and `CoapplicantIncome`.

In [0]:
# Create TotalIncome column and apply a log transformation
df['TotalIncome'] = df_original['ApplicantIncome'] + df_original['CoapplicantIncome']
df[['TotalIncome']] = robust_scaler.fit_transform(df[['TotalIncome']])
df[['TotalIncome']].min()

In [0]:
df['TotalIncome'] = np.log(df['TotalIncome']+2.19)

Let's plot `TotalIncome`.

In [0]:
df['TotalIncome'].hist(bins=20)
plt.show()

Let's drop `CoapplicantIncome`.

In [0]:
df.drop('CoapplicantIncome', axis=1, inplace=True)

# G. Save the Prepared Dataset

Finally, let's save the prepared dataset as a CSV file.

In [0]:
df.to_csv('loans_train_prepared.csv')